# Búsqueda de texto completo
Autor: Eric S. Tellez <eric.tellez@infotec.mx> <br/>

Tal vez la tarea más emblemática de la Recuperación de Información es la búsqueda de _texto completo_.
El problema consiste en dado un corpus grande de documentos, preprocesarlo para crear una estructura de búsqueda que permita resolver consultas de manera eficiente. Una consulta es un texto corto que específica lo que se desea encontrar en la colección. En particular, es un ejemplo de lo que se desea. Esto lleva a que la estructura de búsqueda resuelve búsquedas por similitud.

La similitud, es entonces un tema central, pero para medirla lo primero es tener una representación de los datos que capture las propiedades deseadas (que serán después evaluadas para medir la similitud). La manera más tradicional de hacerlo, es el uso de un modelo basado en bolsa de palabras (BOW). En dicho modelo, el texto es preprocesado, toquenizado y vectorizado.

- El preprocesamiento incluye tratamientos tan simples como eliminar símbolos no deseados, eliminiación de variantes léxicas, reducción a raíces o lemas, corrección de ortografía, eliminiación de palabras comunes (stop words). 
- El toquenizado es el proceso donde el texto es partido, en frases u oraciones, y finalmente en palabras y símbolos que son unidades completas. En este punto también es posible realizar normalizaciones, así como también realizar limpieza basada en estadísticas de los términos.
- El vectorizado utiliza el vocabulario de una colección $\{t_i\}$ para generar una matriz de la colección, i.e., un vector por documento.

Al proceso de modelar una colección mediante un vocabulario y luego ser capaces de generar una representación manejable por una computadora se le llama _modelo de lenguaje_.

## Problema de búsqueda
Una vez que se genero el modelo de lenguaje y que fue usado para vectorizar una colección $X$, la idea es ser capaces de resolver consultas $Q$, i.e., encontrar un subconjunto de $X$ que mejor se apegue a una especificación $q \in Q$. Las consultas deben ser codificadas de la misma forma, para generar un vector con ellas. Entonces el problema se transforma en encontrar los elementos más parecidos, que dada la representación, es conveniente usar el coseno entre vectores:

$$ \cos(u, q) = \frac{ \sum_i {u_i \cdot q_i}}{\sqrt{\sum_i u_i^2} \cdot \sqrt{\sum_i q_i^2}} $$

Así mismo, $d(u, q) = \arccos(\cos(u, q))$ sería el ángulo entre ambos vectores, que además es una métrica. El problema entonces se transforma en encontrar los vecinos más cercanos en la colección, esto es, si deseamos $k$ resultados de una consulta, estaríamos deseando encontrar aquel subconjunto $knn$ de la colección tal que $\sum_{v \in knn} d(v, q)$ sea mínimo comparado con todo subconjunto de tamaño $k$ de la colección de documentos.

## Velocidad de consultas
Para mejorar la solución de consultas, es posible crear una estructura de datos que simplifique el proceso de encontrar el subconjunto $knn$. En este problema, con una representación basada en bolsa de palabras, la estructura más adecuada es el _índice invertido.


# Índice invertido

Un índice invertido es una representación dispersa de la matriz $W_{m,n}$ formada por $m$ componentes y $n$ documentos, i.e., cada celda $w_{t,i}$ es el peso asignado para el término $t$ que ocurre en el documento $i$. Por construcción, esta matriz tiene una gran cantidad de ceros, por lo que $W$ es altamente dispersa (pocos términos ocurren en un documento).

$$ W \left \{
\begin{array}{rrrr rrrr rr}
                & \vec x_1& \vec x_2&       & \vec x_n \\
t_1 \rightarrow & w_{1,1} & w_{1,2} & \dots & w_{1,n} \\
t_2 \rightarrow & w_{2,1} & w_{2,2} &       & w_{2,n} \\
                & \vdots  &         & \ddots&         \\
t_m \rightarrow & w_{m,1} & w_{m,2} &       & w_{m,n} \\
\end{array}
\right .
$$

La representación es entonces por fila, a manera de lista de adjacencia; esto es, cada fila $t$ es representada por las tuplas $(i, w_{t,i})$, esto es, un índice invertido es la siguiente estructura $W^*$

$$ W^* \left \{
\begin{array}{rrr}
t_1 & \rightarrow & \{(i, w_{1, i})\} \\
t_2 & \rightarrow & \{(i, w_{2, i})\} \\
\vdots & \vdots   &  \hfill \vdots \hfill \\
t_m & \rightarrow & \{(i, w_{m, i})\} \\
\end{array}
\right .
$$

la tupla es usada siempre y cuando $w > 0$. Las tuplas suelen ordenarse por su identificador de columna, pero también puede usarse el peso según convenga. A las filas suele llamarseles listas de posteo (_posting lists_). Los requerimientos de una matriz densa son altísimos para representaciones de texto de alta dimensión, representar las matrices de manera dispersa simplifica el manejo de la memoría, y como se verá a continuación, también influye enormemente en los tiempos de procesamiento.

### Búsqueda mediante un índice invertido

La solución na\"ive de una obtener los $k$ documentos más similares es evaluar todos los vectores $\vec{x}_i$, i.e., columnas de $W$, y determinar aquellos más similares, i.e., minimizar $d(\vec{x}_i, q)$.

El índice invertido $W^*$ contiene la información necesaria para realizar esta operación de manera eficiente. Primeramente, es necesario analizar la expresión de $\cos$. El denominador $\sqrt{\sum_i u_i^2} \cdot \sqrt{\sum_i q_i^2}$, en sus partes es estático para cada vector, por lo que se puede preprocesar y no calcular de manera explícita para cada evaluación de $\cos$. Con respecto al numerador corresponde al producto punto entre $\vec{u}$ y $\vec{q}$, $\sum_i u_i \cdot q_i$. Dicho esto, solo es necesario calcular los productos diferentes de cero; así pues, la evaluación eficiente de $\cos$ corresponde con una evaluación eficiente de la intersección de las componentes diferentes de cero. Los algoritmos como SvS, BY o BK, pueden ser de gran ayuda para este cálculo. Note que aunque que los pesos con valor cero no se representan en $W^*$, dicho índice representa información por fila, lo cual no permite hacer operaciones eficientes entre $q$ y los vectores columna $\vec x$ individuales.


Afortunadamente, la evaluación se puede hacer eficiente para todo el conjunto de posibles candidatos (aquellos donde el producto punto contra $q$ sea diferente de cero). Para esto, se toman las componentes diferentes de cero en $q$, se toman las listas de adyacencia de $W^*$ y se procede a unirlas de manera eficiente. El conjunto de identificadores de documento resultado de esta unión será aquel que debe ser evaluado para obtener el conjunto de documentos similares. 
Si uno toma la intersección, que puede ser más veloz de calcular, entonces podrían perderse documentos relavantes; es posible también mandar el problema a un punto intermedio, es decir al problema de $t$-thresholds, donde se recupera un conjunto donde cada uno de los miembros aparece en al menos $t$ listas.
La manera más eficiente, sin embargo, es realizar optimizaciones por filtrado de pesos o mejorando los esquemas de pesado; la idea general entonces es desaparecer entradas de $W*$ de tal forma que la unión sea siempre pequeña. La adecuada optimización de un índice invertido puede hacerlo escalable a niveles realmente impresionantes.

Los algoritmos de BK pueden ser utilizados para calcular la unión y t-threasholds, así como los algoritmos de mezcla clásicos (_merge_). Es posible unir la operación de unión con la operación de producto punto por vector usando los algoritmos adecuados.

- <https://github.com/sadit/InvertedFiles.jl/blob/main/src/invfilesearch.jl>
- <https://github.com/sadit/InvertedFiles.jl/blob/main/src/winvfilesearch.jl>
- <https://github.com/sadit/Intersections.jl/blob/main/src/merge.jl>

# Ejemplo

El siguiente es un ejemplo de como cambia el desempeño usando una evaluación exhaustiva, un índice invertido y una pequeña optimización basada en modificación de pesos. 

In [1]:
using Pkg
Pkg.activate(".")

!isfile("Manifest.toml") && Pkg.add([
    PackageSpec(name="SimilaritySearch", version="0.9"),
    PackageSpec(name="TextSearch", version="0.12"),
    PackageSpec(name="InvertedFiles", version="0.4"),
    PackageSpec(name="CodecZlib", version="0.7"),
    PackageSpec(name="JSON", version="0.21"),
    PackageSpec(name="HypertextLiteral", version="0.9")
])

using TextSearch, InvertedFiles, SimilaritySearch, TextSearch, CodecZlib, JSON, LinearAlgebra, HypertextLiteral


  Activating project at `~/IR-2022/Unidades`


## Funciones para leer y modelar el corpus

In [2]:
function parse_corpus(corpusfile)
    corpus, labels = String[], String[]
    open(corpusfile) do f
        for line in eachline(GzipDecompressorStream(f))
            r = JSON.parse(line)
            push!(labels, r["klass"])
            push!(corpus, r["text"])    
        end
    end
    
    corpus, labels
end

function text_model_and_vectors(
        corpus;
        textconfig=TextConfig(group_usr=true, group_url=true, del_diac=true, lc=true, group_num=true, nlist=[1], qlist=[]),
        model=VectorModel(IdfWeighting(), TfWeighting(), textconfig, corpus)
    )
    vectors = vectorize_corpus(model, textconfig, corpus)
    for v in vectors
        normalize!(v)
    end

    (; textconfig, model, vectors)
end

function create_dataset(corpusfile)
    corpus, labels = parse_corpus(corpusfile)
    (; labels, corpus, text_model_and_vectors(corpus)...)
end

create_dataset (generic function with 1 method)

In [3]:
display(@htl "<h1>Cargando el corpus (descargado en U1.ipynb)</h1>")
dbfile = "../data/emo50k.json.gz"
D = create_dataset(dbfile)
@show unique(D.labels), D.model

<h1>Cargando el corpus (descargado en U1.ipynb)</h1>

(unique(D.labels), D.model) = (["😰", "😥", "😊", "😏", "♡", "💔", "🙂", "😋", "😌", "🌚", "👌", "😪", "😤", "🙃", "🤤", "😴", "😢", "😅", "😑", "😠", "😂", "😜", "🤓", "💙", "😀", "🤗", "🤣", "😒", "✨", "😐", "😞", "😁", "😱", "👏", "😫", "😍", "❤", "😣", "🙊", "🙏", "🙄", "🤭", "💜", "🤔", "😬", "👀", "😉", "😈", "😡", "😳", "🙈", "😻", "😔", "😓", "💕", "🎶", "😭", "😕", "♥", "💖", "😎", "😘", "😃", "😩"], {VectorModel global_weighting=IdfWeighting(), local_weighting=TfWeighting(), train-voc=45374, train-n=50000, maxoccs=93991})


(["😰", "😥", "😊", "😏", "♡", "💔", "🙂", "😋", "😌", "🌚"  …  "💕", "🎶", "😭", "😕", "♥", "💖", "😎", "😘", "😃", "😩"], {VectorModel global_weighting=IdfWeighting(), local_weighting=TfWeighting(), train-voc=45374, train-n=50000, maxoccs=93991})

# Creando los diferentes métodos de búsqueda

In [4]:
# índice invertido
invfile = WeightedInvertedFile(length(D.model.voc))
append!(invfile, VectorDatabase(D.vectors))

## indice invertido con filtros al vocabulario
fmodel = filter_tokens(D.model) do t
    10 <= t.ndocs <= 1000  # filtrando los tokens que ocurren en entre 10 y 1000 documentos
end

## usando el modelo reducido de tokens
fD = text_model_and_vectors(D.corpus, textconfig=D.textconfig, model=fmodel)

finvfile = WeightedInvertedFile(length(fD.model.voc))
append!(finvfile, VectorDatabase(fD.vectors))

{WeightedInvertedFile vocsize=3968, n=50000}

In [5]:
display(@htl "The token filters can operate on any valid field of voc (passed as the named tuple <em>t</em>)")

dump(typeof(D.model.voc))

The token filters can operate on any valid field of voc (passed as the named tuple <em>t</em>)

Vocabulary <: Any
  token::Vector{String}
  occs::Vector{Int32}
  ndocs::Vector{Int32}
  weight::Vector{Float32}
  token2id::Dict{String, UInt32}
  corpuslen::Int64


In [6]:
Q = VectorDatabase(rand(D.vectors, 1000))
fQ = VectorDatabase(rand(fD.vectors, 1000))

VectorDatabase{Vector{Dict{UInt32, Float32}}}(Dict{UInt32, Float32}[Dict(0x00000054 => 0.3347092, 0x0000068a => 0.6846173, 0x00000044 => 0.34018824, 0x00000419 => 0.5509455), Dict(0x00000186 => 0.7395449, 0x000000a2 => 0.6731072), Dict(0x000000b0 => 0.23381734, 0x00000053 => 0.3259642, 0x000000e7 => 0.3169817, 0x0000001b => 0.2925202, 0x00000d35 => 0.49764267, 0x000001f5 => 0.34775817, 0x00000cb9 => 0.42758507, 0x000000e8 => 0.3187763), Dict(0x000000b0 => 0.3558882, 0x00000eef => 0.74962527, 0x000002e8 => 0.5580372), Dict(0x0000016b => 0.2604656, 0x0000004f => 0.23028766, 0x00000448 => 0.4283243, 0x000000c9 => 0.26923087, 0x000003eb => 0.44006115, 0x00000365 => 0.3291022, 0x000007af => 0.42488283, 0x00000a58 => 0.37508774), Dict(0x00000259 => 0.39137, 0x00000dd7 => 0.7243029, 0x000000bb => 0.5676397), Dict(0x00000084 => 0.4996388, 0x00000d45 => 0.8662338), Dict(0x00000487 => 0.22856358, 0x00000bf3 => 0.2630118, 0x000006a7 => 0.23114121, 0x00000293 => 0.2616991, 0x00000916 => 0.25267273

In [7]:
# evaluación exhaustiva
brute = ExhaustiveSearch(; db=D.vectors, dist=NormalizedCosineDistance())
fbrute = ExhaustiveSearch(; db=fD.vectors, dist=NormalizedCosineDistance())

@time searchbatch(brute, Q, 16)
@time searchbatch(invfile, Q, 16)
@time searchbatch(fbrute, fQ, 16)
@time searchbatch(finvfile, fQ, 16)
nothing

  1.292747 seconds (1.93 M allocations: 141.872 MiB, 3.65% gc time, 65.59% compilation time)
  0.491890 seconds (648.26 k allocations: 34.584 MiB, 67.35% compilation time)
  0.239561 seconds (21 allocations: 125.688 KiB)
  0.003348 seconds (6 allocations: 125.250 KiB)


## Se volverá a realizar la operación pero con unas pequeñas modificaciones para remover el costo de compilación y el costo de recolección de basura

In [8]:
GC.enable(false)
@time I, _ = searchbatch(invfile, Q, 16)
@time G, _ = searchbatch(brute, Q, 16)
@time fI, _ = searchbatch(finvfile, fQ, 16)
@time fG, _= searchbatch(fbrute, fQ, 16)
GC.enable(true)

  0.164695 seconds (2.69 k allocations: 280.063 KiB, 7.34% compilation time)
  0.484623 seconds (70 allocations: 127.219 KiB)
  0.003244 seconds (8 allocations: 125.312 KiB)
  0.238250 seconds (22 allocations: 125.719 KiB)


false

In [9]:
### La diferencia de costos puede ser importante, por lo que es también necesario saber el impacto con respecto a la calidad perdida
@show macrorecall(G, I)
@show macrorecall(fG, fI)
@show macrorecall(G, fG)
@show macrorecall(G, fI)

macrorecall(G, I) = 0.9975
macrorecall(fG, fI) = 0.9498125
macrorecall(G, fG) = 0.001
macrorecall(G, fI) = 0.0009375


0.0009375

# Sobre los resultados
Afortunadamente no es tan simple como decir que se destruyo la calidad del resultado,
ya que eso implicaría que el modelo sin filtros fuera mejor y eso no es necesariamente
cierto ya que los filtros podrían estar removiendo términos _ruidosos_ (errores ortográficos y palabras que al ser tan comunes aportan poco al resultado).
Lo que sí se puede afirmar es que los dos conjuntos de resultados son diferentes y esto es importante de tener
en cuenta cuando se realizan modificaciones a los modelos.

Tradicionalmente, los resultados se miden con el recall, pero el _gold standard_ es construido a partir de
intervención humana, que es capaz de decir que documentos son relevantes a una consulta.

# Actividades

- ¿Qué es el recall?
- ¿Qué es macro-recall?
- Explique el porque de las mejoras en tiempo. Use análisis de algoritmos para este ejercicio.
- Implemente un índice invertido que sea capaz de mejorar los tiempos de búsqueda tal y como se muestra en este ejercicio. Para el procesamiento del texto y toquenizado use librerias/paquetes externos.
- Reporte mediante un notebook de Jupyter su implementación, use uno de los conjuntos de datos para ejemplificar su uso y medir los desempeños.